In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Final.csv')

In [3]:
df.head()

,Price,Year,Mileage,City,State,Make,Model
0,16472,2015,18681,JEFFERSON CITY,MO,BUICK,ENCORECONVENIENCE
1,15749,2015,27592,HIGHLAND,IN,BUICK,ENCOREFWD
2,16998,2015,13650,BOONE,NC,BUICK,ENCORELEATHER
3,15777,2015,25195,NEW ORLEANS,LA,BUICK,ENCOREFWD
4,16784,2015,22800,LAS VEGAS,NV,BUICK,ENCORECONVENIENCE


In [4]:
import sklearn
print(sklearn.__version__)

1.1.0


In [5]:
X = df.drop('Price',axis =1)
y = df['Price']

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [8]:
ohe=OneHotEncoder()
ohe.fit(X[['City', 'State', 'Make', 'Model']])

OneHotEncoder()

In [9]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['City', 
                                                                                 'State', 
                                                                                 'Make', 
                                                                                 'Model']),
                                     remainder='passthrough')

In [10]:
from xgboost import XGBRegressor

In [11]:
model = XGBRegressor(n_estimators = 1000)

In [12]:
pipe=make_pipeline(column_trans,model)

In [13]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['ABBEVILLE', 'ABERDEEN', 'ABILENE', ..., 'ZELIENOPLE',
       'ZEPHYRHILLS', 'ZIONSVILLE'], dtype=object),
                                                                            array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME...
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0,
                              reg_lambda=1, ...))])

In [14]:
y_pred=pipe.predict(X_test)

In [15]:
r2_score(y_test,y_pred)

0.9117266675905691

In [16]:
input_df = pd.DataFrame({'Make':['BUICK'],'Model':['ENCORECONVENIENCE'],'Year':[2017],'State':['MO'],'City':['JEFFERSON CITY'],'Mileage':[2000]})

In [17]:
result = pipe.predict(input_df)

In [18]:
print(result)

[24414.762]


In [19]:
import pickle
pickle.dump(pipe,open('xgboostmodel.pkl','wb'))